In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [2]:
!pip install fsspec==2023.9.0 datasets==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
import evaluate
from datasets import load_dataset, concatenate_datasets
import pandas as pd

In [7]:
data_files = {
    "fold_1": "folds/fold_1.csv", 
    "fold_2": "folds/fold_2.csv",
    "fold_3": "folds/fold_3.csv",
    "fold_4": "folds/fold_4.csv",
    "fold_5": "folds/fold_5.csv",
    "fold_6": "folds/fold_6.csv"
}
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating fold_1 split: 0 examples [00:00, ? examples/s]

Generating fold_2 split: 0 examples [00:00, ? examples/s]

Generating fold_3 split: 0 examples [00:00, ? examples/s]

Generating fold_4 split: 0 examples [00:00, ? examples/s]

Generating fold_5 split: 0 examples [00:00, ? examples/s]

Generating fold_6 split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-36820d5847e8ecc9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
train_dataset = concatenate_datasets([
    dataset['fold_1'],
    dataset['fold_2'],
    dataset['fold_3'],
    dataset['fold_4'],
    dataset['fold_5']
])

eval_dataset = dataset['fold_6']

In [6]:
def extract_feature(path):
    with soundfile.SoundFile(path) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    return mfccs

In [9]:
def preprocess_function(data):
    speech_list = [extract_feature(path) for path in data["path"]]
    target_list = [label-1 for label in data["emotion"]]

    return speech_list, target_list

In [10]:
features_train, targets_train = preprocess_function(train_dataset)
features_eval, targets_eval = preprocess_function(eval_dataset)

In [11]:
def compute_metrics(preds, eval_test):
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    
    results = {}
    results.update(accuracy_metric.compute(predictions=preds, references=eval_test))
    results.update(f1_metric.compute(predictions=preds, references=eval_test, average='micro'))
    results.update(precision_metric.compute(predictions=preds, references=eval_test, average='micro'))
    results.update(recall_metric.compute(predictions=preds, references=eval_test, average='micro'))
    return results

In [12]:
reg_coefs = [0.05, 0.1, 0.5, 1, 5]
kernels = ['linear', 'poly']
gammas = [0.01, 0.05, 0.1, 0.5, 1]

In [13]:
metrics = []
for kernel in kernels:
    for reg_coef in reg_coefs:
        for gamma in gammas:
            model = svm.SVC(C=reg_coef, 
                            kernel=kernel, 
                            degree=2, 
                            gamma=gamma,
                            tol=0.001)
            model.fit(features_train, targets_train)
            preds = model.predict(features_eval)
            metrics.append((
                kernel, reg_coef, gamma, compute_metrics(preds, targets_eval)
            ))
            print("Done")
            

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [14]:
metrics

[('linear',
  0.05,
  0.01,
  {'accuracy': 0.3, 'f1': 0.3, 'precision': 0.3, 'recall': 0.3}),
 ('linear',
  0.05,
  0.05,
  {'accuracy': 0.3, 'f1': 0.3, 'precision': 0.3, 'recall': 0.3}),
 ('linear',
  0.05,
  0.1,
  {'accuracy': 0.3, 'f1': 0.3, 'precision': 0.3, 'recall': 0.3}),
 ('linear',
  0.05,
  0.5,
  {'accuracy': 0.3, 'f1': 0.3, 'precision': 0.3, 'recall': 0.3}),
 ('linear',
  0.05,
  1,
  {'accuracy': 0.3, 'f1': 0.3, 'precision': 0.3, 'recall': 0.3}),
 ('linear',
  0.1,
  0.01,
  {'accuracy': 0.32083333333333336,
   'f1': 0.32083333333333336,
   'precision': 0.32083333333333336,
   'recall': 0.32083333333333336}),
 ('linear',
  0.1,
  0.05,
  {'accuracy': 0.32083333333333336,
   'f1': 0.32083333333333336,
   'precision': 0.32083333333333336,
   'recall': 0.32083333333333336}),
 ('linear',
  0.1,
  0.1,
  {'accuracy': 0.32083333333333336,
   'f1': 0.32083333333333336,
   'precision': 0.32083333333333336,
   'recall': 0.32083333333333336}),
 ('linear',
  0.1,
  0.5,
  {'accuracy

One can see that the best model is $kernel=linear, C=1, gamma=1$. We've got $32,5$% accuracy (random classifier gives us 12.5%)